In [5]:

import pandas as pd
import plotly.express as px
from scipy.signal import find_peaks, peak_widths

filename = "spectra.txt"
peak_prominance = 1_000
peak_prominance = 10_000
peak_prominance = 65_000

filename = 'asymmetric_spectrum_data.csv'
# filename = 'symmetric_data.csv'
peak_prominance = .6

data = pd.read_csv(filename, sep='\s+')


def find_assymetry(data, peak_prominance, filename):
    fig = px.line(data, x="wave_nm", y="int", title=f'Spectra {filename}')
    peaks, properties = find_peaks(data["int"], prominence=peak_prominance)

    for i, peak in enumerate(peaks):
        widths, _, left_ips, right_ips = peak_widths(data["int"], peaks, rel_height=0.95)
        # w_005 = widths[i] + left_ips[i]
        # w_005 = right_ips[i]
        # f = left_ips[i]
        # symmetry_factor = w_005 / (1 * f)
        
        center = (right_ips[i] + left_ips[i]) / 2
        symmetry_factor = (right_ips[i] - center) / (center - left_ips[i])

        fig.add_annotation(x=data['wave_nm'].iloc[peak],
                           y=data['int'].iloc[peak],
                           text=f'S: {symmetry_factor:.3f}',
                           showarrow=True,
                           arrowhead=2)

        # fig.add_vrect(x0=data['wave_nm'].iloc[right_ips[i]], x1=data['wave_nm'].iloc[left_ips[i]], 
        #       annotation_text="decline", annotation_position="top left",
        #       fillcolor="green", opacity=0.25, line_width=0)
        fig.write_html(f'spectra_{peak_prominance}_{filename}.html')
    return fig

# Plot original spectra and find asymmetry
fig = find_assymetry(data, peak_prominance, filename)
fig.show()


In [9]:
import pandas as pd
import plotly.express as px
from scipy.signal import find_peaks, peak_widths
import numpy as np

def find_asymmetry(data, peak_prominence, filename):
    fig = px.line(data, x="wave_nm", y="int", title=f'Spectra {filename}')
    peaks, properties = find_peaks(data["int"], prominence=peak_prominence)

    for i, peak in enumerate(peaks):
        widths, heights, left_ips, right_ips = peak_widths(data["int"], [peak], rel_height=0.95)
        
        # Convert interpolated indices to x-axis values
        left_x = np.interp(left_ips[0], range(len(data)), data['wave_nm'])
        right_x = np.interp(right_ips[0], range(len(data)), data['wave_nm'])
        peak_x = data['wave_nm'].iloc[peak]
        
        # Calculate asymmetry factor
        left_half_width = peak_x - left_x
        right_half_width = right_x - peak_x
        asymmetry_factor = right_half_width / left_half_width
        
        print(f"Peak {i+1}: Left HW = {left_half_width:.2f}, Right HW = {right_half_width:.2f}, Asymmetry Factor = {asymmetry_factor:.3f}")

        # Add annotation to the plot
        fig.add_annotation(x=peak_x, y=data['int'].iloc[peak],
                           text=f'AF: {asymmetry_factor:.3f}',
                           showarrow=True,
                           arrowhead=2)
        
        # Add vertical lines for peak boundaries
        fig.add_vline(x=left_x, line_dash="dash", line_color="red")
        fig.add_vline(x=right_x, line_dash="dash", line_color="red")

    fig.write_html(f'spectra_{peak_prominence}_{filename}.html')
    return fig

# Usage
filename = 'asymmetric_data.csv'  # Replace with your actual filename
peak_prominence = 0.6  # Adjust as needed
data = pd.read_csv(filename, sep='\s+')
fig = find_asymmetry(data, peak_prominence, filename)
fig.show()

Peak 1: Left HW = 12.55, Right HW = 24.19, Asymmetry Factor = 1.928
Peak 2: Left HW = 19.44, Right HW = 36.87, Asymmetry Factor = 1.896
Peak 3: Left HW = 14.85, Right HW = 29.23, Asymmetry Factor = 1.968
